# [Module 2.4] 인퍼런스 시간 벤치 마킹

### 본 워크샵의 모든 노트북은 `conda_python3` 여기에서 작업 합니다.

이 노트북은 아래와 같은 작업을 합니다.

- 0. 개념 확인
- 1. 환경 셋업
- 2. 모델 아티펙트 다운로드 및 압축해제
- 3. 추론 함수 로컬 테스트
- 4. 로컬 엔드포인트 생성
- 5. 로컬 추론
- 6. 로컬 엔드 포인트 삭제


### 참고: 
- 세이지 메이커 개발자 가이드 --> [추론을 위한 모델 배포](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/deploy-model.html)
- 세이지 메이커 배포에 대한 웹비나 --> [Amazon SageMaker 기반 사전 훈련된 딥러닝 모델 손쉽게 배포하기 – 김대근:: AWS Innovate 2021](https://www.youtube.com/watch?v=ZdOcrLKow3I)
- 세이지 메이커 호스팅 기본 컨셉 --> [SageMaker 호스팅 아키텍쳐](https://github.com/gonsoomoon-ml/SageMaker-Pipelines-Step-By-Step/blob/main/scratch/8.1.Deploy-Pipeline.ipynb)
- 세이지 메이커로 파이토치 사용 --> [Use PyTorch with the SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/using_pytorch.html)
    
---    
    

# 0. 개념 확인

# 1. 환경 셋업

## 1.1. 기본 세팅
사용하는 패키지는 import 시점에 다시 재로딩 합니다.

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('./src')

전 노트북에서 훈련 후의 아티펙트를 가져옵니다.

In [2]:
%store -r artifact_path
%store -r bucket
%store -r prefix

In [3]:
import sagemaker

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

## 1.2. 배포 준비

### 1.2.1. 모델 아티펙트 확인
이 워크샵에서 제공한 노트북을 실행하셨다고 하면 아래의 모델 아티펙트가 생성이 되었을 겁니다.
artifact_path 에 해당 변수를 할당하셔서 사용하시면 됩니다.



In [4]:
print("model artifact is assigend from : ", artifact_path)

model artifact is assigend from :  s3://sagemaker-ap-northeast-2-057716757052/pytorch-training-2023-03-05-09-10-14-368/output/model.tar.gz


### 1.2.2. 추론을 위한  데이터 세트 로딩
- 전부 데이터를 로딩할 필요가 없지만, 여기서는 기존에 사용한 함수를 이용하기 위해서 전체 데이터를 로드 합니다. 
    - 실제 데이터로 구현시에는 따로이 로드 함수를 사용하시기를 권장 합니다.


In [5]:
import data_utils 
train_data, test_data, user_num ,item_num, train_mat = data_utils.load_all(test_num=100)

In [6]:
class Params:
    def __init__(self):
        # self.epochs = 1        
        self.num_ng = 4
        self.batch_size = 256
        self.test_num_ng = 99
        self.factor_num = 32
        self.num_layers = 3
        self.dropout = 0.0
        # self.lr = 0.001
        self.top_k = 10
        self.out = True
        # self.gpu = "0"
                        
args = Params()
print("# of batch_size: ", args.batch_size)

import torch.utils.data as data

test_dataset = data_utils.NCFData(
		test_data, item_num, train_mat, 0, False)

test_loader = data.DataLoader(test_dataset,
		batch_size=args.test_num_ng+1, shuffle=False, num_workers=0)



# of batch_size:  256


# 2. 모델 아티펙트 다운로드 및 압축해제
- 모델 아티펙트를 다운로드 합니다.
- 다운로드 받은 모델 아티펙트의 압축을 해제하고 모델 가중치인 model.pth 파일을 얻습니다.

In [7]:
import os
import config

model_data_dir = config.model_path
os.makedirs(model_data_dir, exist_ok=True)
print("model_data_dir: ", model_data_dir)

model_data_dir:  ./models/


In [8]:
%%sh -s {artifact_path} {model_data_dir}

artifact_path=$1
model_data_dir=$2

echo $artifact_path
echo $model_data_dir

# 기존 데이터 삭제
rm -rf $model_data_dir/*

# 모델을 S3에서 로컬로 다운로드
aws s3 cp $artifact_path $model_data_dir

# 모델 다운로드 폴더로 이동
cd $model_data_dir

# 압축 해제
tar -xvf model.tar.gz  

s3://sagemaker-ap-northeast-2-057716757052/pytorch-training-2023-03-05-09-10-14-368/output/model.tar.gz
./models/
download: s3://sagemaker-ap-northeast-2-057716757052/pytorch-training-2023-03-05-09-10-14-368/output/model.tar.gz to models/model.tar.gz
NeuMF-end.pth


# 3. 추론 함수 로컬 테스트


## 3.1. 추론시 사용할 모델 네트워크 설정 저장
- 모델 네트워크를 생성시에 설정값을 model_config.json 로 저장함.
- model_fn() 함수에서 모델 네트워크를 생성시에 사용 함.

In [9]:
import json
from common_utils import save_json, load_json

model_config_dict = {
    'user_num': str(user_num),
    'item_num': str(item_num),
    'factor_num' : str(args.factor_num),
    'num_layers' : str(args.num_layers),
    'dropout' : str(args.dropout),
    'model_type': config.model
}

model_config_file = 'model_config.json'
model_config_file_path = os.path.join('src', model_config_file)

save_json(model_config_file_path, model_config_dict)
# model_config_dict = load_json(model_config_file_path)    
# model_config_dict

src/model_config.json is saved


'src/model_config.json'

## 3.2. 사용자 정의 inference code

- 사용자 정의 inference 코드를 정의해서 사용하기 전에, 노트북에서 사전 테스트 및 디버깅을 하고 진행하면 빠르게 추론 개발을 할수 있습니다.


- 디폴트 inference code (input_fn, model_fn, predict_fn, output_fn) 을 사용해도 되지만, 상황에 따라서는 사용자 정의가 필요할 수 있습니다. 디폴트 코드는 아래 링크를 참고 하세요.
    - [Deploy PyTorch Models](https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/using_pytorch.html#deploy-pytorch-models)
    - [디폴트 inference Code](https://github.com/aws/sagemaker-pytorch-inference-toolkit/blob/master/src/sagemaker_pytorch_serving_container/default_pytorch_inference_handler.py)





In [10]:
from inference import model_fn , input_fn, predict_fn


### 3.2.1. model_fn 테스트
- 훈련한 모델 아티펙트 파일이 정상적으로 모델에 로딩이 되는지를 확인 합니다.
- 일반적인 모델 로딩 에러
    - Torch Version(에: 1.6.0) 이 훈련시에 사용한 버전과 동일해야 합니다. 만약 다들 경우에 `RuntimeError: version_ <= kMaxSupportedFileFormatVersion INTERNAL ASSERT FAILED` 에러가 발생합니다.
    - 훈련시에 사용한 리소스 (GPU or CPU)가 추론시에도 동일해야 합니다.

In [11]:
inf_model = model_fn(config.model_path)

######## Staring model_fn() ###############
--> model_dir : ./models/
model_config_path: :  /home/ec2-user/SageMaker/aws-ai-ml-workshop-kr/sagemaker/recommendation/Neural-Collaborative-Filtering-On-SageMaker/2_Inference/./src/model_config.json
--> model network is loaded
model_file_path: :  {model_file_path}
---> ########## Failure loading a Model #######
Traceback (most recent call last):
  File "/home/ec2-user/SageMaker/aws-ai-ml-workshop-kr/sagemaker/recommendation/Neural-Collaborative-Filtering-On-SageMaker/2_Inference/./src/inference.py", line 85, in model_fn
    inf_model.load_state_dict(torch.load(f))
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1604, in load_state_dict
    raise RuntimeError('Error(s) in loading state_dict for {}:\n\t{}'.format(
RuntimeError: Error(s) in loading state_dict for NCF:
	Unexpected key(s) in state_dict: "linear.weight", "linear.bias". 



### 3.2.2. input_fn 테스트
- 추론시의 입력을 input_fn에 전달하고 결과값이 잘 나오는지를 확인 합니다.
- 입력값의 형태에 따라서 input_fn이 변경이 필요할 수 있습니다.


추론 테스트할 1개의 레코드를 가져옵니다.

In [12]:
for user, item, label in test_loader:   
    user_np = user.detach().cpu().numpy()
    item_np = item.detach().cpu().numpy()            
    break
payload = {'user':user_np.tolist(), 'item':item_np.tolist()}
print("payload: ", payload)


payload:  {'user': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'item': [25, 1064, 174, 2791, 3373, 269, 2678, 1902, 3641, 1216, 915, 3672, 2803, 2344, 986, 3217, 2824, 2598, 464, 2340, 1952, 1855, 1353, 1547, 3487, 3293, 1541, 2414, 2728, 340, 1421, 1963, 2545, 972, 487, 3463, 2727, 1135, 3135, 128, 175, 2423, 1974, 2515, 3278, 3079, 1527, 2182, 1018, 2800, 1830, 1539, 617, 247, 3448, 1699, 1420, 2487, 198, 811, 1010, 1423, 2840, 1770, 881, 1913, 1803, 1734, 3326, 1617, 224, 3352, 1869, 1182, 1331, 336, 2517, 1721, 3512, 3656, 273, 1026, 1991, 2190, 998, 3386, 3369, 185, 2822, 864, 2854, 3067, 58, 2551, 2333, 2688, 3703, 1300, 1924, 3118]}


In [13]:
input_fn_payload = input_fn(json.dumps(payload), content_type='application/json')
print("input_fn_payload: ", input_fn_payload)

#### input_fn starting ######
content_type: application/json
#### type of input data: <class 'str'>
input_fn_payload:  [tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0], device='cuda:0'), tensor([  25, 1064,  174, 2791, 3373,  269, 2678, 1902, 3641, 1216,  915, 3672,
        2803, 2344,  986, 3217, 2824, 2598,  464, 2340, 1952, 1855, 1353, 1547,
        3487, 3293, 1541, 2414, 2728,  340, 1421, 1963, 2545,  972,  487, 3463,
        2727, 1135, 3135,  128,  175, 2423, 1974, 2515, 3278, 3079, 1527, 2182,
        1018, 2800, 1830, 1539,  617,  247, 3448, 1699, 1420, 2487,  198,  811,
        1010, 1423, 2840, 1770,  881, 1913, 1803, 1734, 3326, 1617,  224, 3352,
        1869, 1182, 1331,  336, 2517, 172

### 3.2.3. predict_fn 테스트
- input_fn 의 결과값이 predict_fn으로 전달이 됩니다. 이에 대한 동작을 확인 합니다.

In [14]:
prediction = predict_fn(input_fn_payload, inf_model)
#output = predict_fn(images, model)
prediction

#### predict_fn starting ######
#### type of input data: <class 'list'>
recommends:  [128, 273, 25, 174, 58, 175, 198, 464, 487, 1064]


[128, 273, 25, 174, 58, 175, 198, 464, 487, 1064]

# 4. 벤치 마킹

In [15]:
import torch

In [16]:
def split_input(input_example):
    user = input_example[0]
    item = input_example[1]    
    
    return user, item

def load_model_on_accelerator(model):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()    
    
    return model


## 일부 샘플로 추론 시간 측정

아래는 예시로서 추론 시간이 0.88 ms, 0.66 ms 걸린다는 것을 의미 합니다.
```
latencies:  [0.88191, 0.66829, 0.63276, 0.77081, 0.61989]
```

In [17]:
import time
latencies = []
num_test = 5
for _ in range(num_test):
    model = load_model_on_accelerator(inf_model)
    user, item = split_input(input_fn_payload)
    start = time.time()
    predictions = model(user, item)    
    finish = time.time()
    elapse_time = round((finish - start) * 1000, 5)
    latencies.append(elapse_time)

print("latencies: ", latencies)

latencies:  [0.88191, 0.66829, 0.63276, 0.77081, 0.61989]


## 벤치 마킹 (ml.p3.2xlarge)

- 스펙
    - instqance = ml.p3.2xlarge
        - GPU - V100 1장
    - throughput = inferences / duration
    - inferences : 인퍼런스 개수
    - duration : 벤치 마크가 총 수행된 초 (단위: second)
    - Latency: P50, P95, P99 - 전체 추론 시간의 퍼센타일 정보
        - (단위: milli-second)
 
 ```
Batch Size:  1
Batches:     1000
Inferences:  1000
Threads:     1
Models:      1
Duration:    0.594
Throughput:  1682.111
Latency P50: 0.585
Latency P95: 0.624
Latency P99: 0.637
 ```

In [24]:
from benchmark_util import benchmark
model = load_model_on_accelerator(inf_model)
user, item = split_input(input_fn_payload)

benchmark(model, user, item, n_models=1, n_threads=1, batches_per_thread=1000)

Batch Size:  1
Batches:     1000
Inferences:  1000
Threads:     1
Models:      1
Duration:    0.594
Throughput:  1682.111
Latency P50: 0.585
Latency P95: 0.624
Latency P99: 0.637
